Cristobal Alcaino


In [3]:
import pandas as pd
df_sc = pd.read_excel("BBDD/SC_COTIZACIONES.xlsx")
print(df_sc.head())

  FECHA_COTIZACION  NUMERO_SOLICITUD  NUMERO_IDENTIFICACION  NUMERO_SECUENCIA  \
0       2024-01-02         125720401              188965131                 1   
1       2024-01-02         125720401              188965150                 1   
2       2024-01-02         125720401              188965169                 1   
3       2024-01-02         125720401              188965188                 1   
4       2024-01-02         125747401              188975930                 1   

  TIPO_RENTA  MESES_DIFERIDOS MODALIDAD_RENTA  MESES_GARANTIZADOS  \
0          I                0               S                   0   
1          I                0               G                 120   
2          I                0               G                 192   
3          I                0               G                 180   
4          I                0               S                   0   

   MESES_AUMENTO_TEMPORAL  PORCENTAJE_AUMENTO_TEMPORAL  ...  PRIMA_UNICA  \
0                     

In [4]:
import pandas as pd
df = pd.read_excel("BBDD/COTIZACIONES.xlsx")
print(df.head())

  FECHA_COTIZACION TIPO_PENSION TIPO_COTIZACION  COTIZANTE  COTIZACION  \
0       2024-01-02            S               E  421008240           1   
1       2024-01-02            S               E  421008240           2   
2       2024-01-02            S               E  421008240           3   
3       2024-01-02            S               E  421008240           4   
4       2024-01-02           VA               E  421005868          11   

  TIPO_INTERMEDIARIO RUT_CORREDOR TIPO_RENTA  MESES_DIFERIDOS MODALIDAD_RENTA  \
0                  S   11370691-0          I                0               S   
1                  S   11370691-0          I                0               G   
2                  S   11370691-0          I                0               G   
3                  S   11370691-0          I                0               G   
4                  S   13982859-3          I                0               G   

   ...        VAN  TASA_VAN  DURATION  DIAS_VALIDEZ  INVALIDA  TIPO_

In [5]:
cotizantes_comunes = pd.merge(df, df_sc, on=['COTIZANTE', 'COTIZACION'], how='inner')
print(f"Cotizantes comunes: {len(cotizantes_comunes)}")

Cotizantes comunes: 962929


In [6]:
cotizantes_comunes[(cotizantes_comunes['COTIZANTE'] == 421007881) & (cotizantes_comunes['COTIZACION'] == 1)]

,FECHA_COTIZACION_x,TIPO_PENSION,TIPO_COTIZACION,COTIZANTE,COTIZACION,TIPO_INTERMEDIARIO,RUT_CORREDOR,TIPO_RENTA_x,MESES_DIFERIDOS_x,MODALIDAD_RENTA_x,...,PRIMA_UNICA_y,RETENCION_AFP_y,RETIRO_EXCEDENTES_y,INVALIDA_y,TIPO_ERROR_y,POSICION_RELATIVA,SEGMENTO,NOMBRE_SEGMENTO,TIT_PROPUESTA,TIT_MAXIMA_y
0,2024-01-02,V,A,421007881,1,NaN,NaN,I,0,S,...,730.59,0.0,0.0,0,0,2,72,V 700-800 UF,3.26,3.28


In [7]:
df_sc[(df_sc['COTIZANTE'] == 421007881) & (df_sc['COTIZACION'] == 1)]

,FECHA_COTIZACION,NUMERO_SOLICITUD,NUMERO_IDENTIFICACION,NUMERO_SECUENCIA,TIPO_RENTA,MESES_DIFERIDOS,MODALIDAD_RENTA,MESES_GARANTIZADOS,MESES_AUMENTO_TEMPORAL,PORCENTAJE_AUMENTO_TEMPORAL,...,PRIMA_UNICA,RETENCION_AFP,RETIRO_EXCEDENTES,INVALIDA,TIPO_ERROR,POSICION_RELATIVA,SEGMENTO,NOMBRE_SEGMENTO,TIT_PROPUESTA,TIT_MAXIMA
0,2024-01-02,125720401,188965131,1,I,0,S,0,0,0,...,730.59,0.0,0.0,0,0,2,72,V 700-800 UF,3.26,3.28


In [8]:
cotizantes_comunes.to_csv('cotizantes_comunes.csv', index=False, encoding='utf-8')  # Sin índice y con UTF-8 para caracteres especiales

In [9]:
import pandas as pd
cotizantes_comunes = pd.read_csv('cotizantes_comunes.csv')

intento de implementacion de regresion logistica

In [10]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [11]:
df_sample = df_sc #.sample(n=100000, random_state=42).copy()

# 1.2 Conversión de columnas numéricas con comas
cols_to_convert = [
    "COTIZACION", "RENTA", "PRIMA_UNICA", 
    "RETENCION_AFP", "RETIRO_EXCEDENTES", 
    "TIT_PROPUESTA", "TIT_MAXIMA", 
    "PORCENTAJE_COMISION"
]

for col in cols_to_convert:
    df_sample[col] = df_sample[col].astype(str).str.replace(',', '.', regex=False).astype(float)

# 1.3 Codificación de variables categóricas
categorical_cols = ["TIPO_RENTA", "MODALIDAD_RENTA", "SEGMENTO", "NOMBRE_SEGMENTO"]
df_sample[categorical_cols] = df_sample[categorical_cols].astype("category")

# One-hot encoding
df_encoded = pd.get_dummies(df_sample, columns=categorical_cols)

# 1.4 Features (X) y etiqueta (y)
X = df_encoded.drop(columns=["POSICION_RELATIVA", "FECHA_COTIZACION"])
y = df_sample["POSICION_RELATIVA"].astype(int)

# Guardar índice original antes de imputar
X["index_original"] = df_sample.index

# Imputación de NaN con la media
imputer = SimpleImputer(strategy="mean")
X_imputed = pd.DataFrame(imputer.fit_transform(X.drop(columns=["index_original"])), columns=X.columns[:-1])
X_imputed["index_original"] = X["index_original"].values

# --- Paso 1.5: Filtrar clases con al menos 2 observaciones ---
conteo_clases = y.value_counts()
clases_validas = conteo_clases[conteo_clases >= 2].index

# Filtrar X e y con las clases válidas
filtro_validas = y.isin(clases_validas)
X_imputed = X_imputed[filtro_validas].reset_index(drop=True)
y = y[filtro_validas].reset_index(drop=True)


In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed.drop(columns=["index_original"]))
X_scaled = pd.DataFrame(X_scaled, columns = X_imputed.columns[:-1])
X_scaled["index_original"] = X_imputed["index_original"].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled.drop(columns = ["index_original"]),
                                                    y,
                                                    test_size=0.2,
                                                    stratify = y,
                                                    random_state=42
)

index_test = X_scaled.loc[X_test.index, "index_original"]

In [14]:
# Entrenamiento
# dudas -> se podra incluir mas data por aca??

logreg = LogisticRegression(
    multi_class = "multinomial",
    solver="lbfgs",
    max_iter=1000, 
    class_weight="balanced",
    random_state=42
)
logreg.fit(X_train, y_train)

# evaluacion (igual)
y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))

c:\Users\Vicente\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      0.98      0.99     79845
           1       0.33      0.67      0.44     15718
           2       0.44      0.35      0.39     29611
           3       0.33      0.16      0.21     24417
           4       0.47      0.16      0.24     20907
           5       0.21      0.18      0.19     12116
           6       0.15      0.22      0.18      6410
           7       0.08      0.22      0.11      2712
           8       0.05      0.36      0.09       710
           9       0.02      0.54      0.03       135
          10       0.00      0.80      0.00         5

    accuracy                           0.58    192586
   macro avg       0.28      0.42      0.26    192586
weighted avg       0.62      0.58      0.58    192586



In [15]:
y_prob = logreg.predict_proba(X_test)

In [17]:
y_prob = logreg.predict_proba(X_test)  
prob_df = pd.DataFrame(y_prob, columns=[f"proba_pos_{cls}" for cls in logreg.classes_])
prob_df["prediccion"] = y_pred
prob_df["real"] = y_test.values
prob_df["index_original"] = index_test.values

original_test_data = df_sample.loc[prob_df["index_original"].values]
resultados_finales = pd.concat([original_test_data.reset_index(drop=True), prob_df.reset_index(drop=True)], axis=1)

resultados_finales.to_csv("predicciones_regresion_logistica.csv", index=False)

XGBOOST

In [ ]:
import pandas as pd
from xgboost import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_imputed.drop(columns=["index_original"]),
    y, 
    test_size=0.2,
    stratify=y,
    random_state=42
)

import re

def clean_feature_names(df):
    df.columns = [re.sub(r'[\[\]<>]', '', col) for col in df.columns]
    return df

X_train = clean_feature_names(X_train)
X_test = clean_feature_names(X_test)

index_test = X_imputed.loc[X_test.index, "index_original"]

xgb_model = XGBClassifier(
    objective = "multi:softmax",
    num_class = len(y.unique()),
    n_estimators = 200,
    max_depth = 6,
    learning_rate = 0.1,
    subsample = 0.8,
    colsample_bytree = 0.8,
    random_state = 42,
    scale_pos_weight = "balanced"
)



In [29]:
xgb_model.fit(X_train, y_train)

ValueError: feature_names must be string, and may not contain [, ] or <